In [1]:
import numpy as np
import pandas as pd


In [2]:
pd.isnull(None)

True

In [4]:
pd.isnull(np.nan)

True

In [5]:
pd.isna(None)

True

In [6]:
pd.isna(np.nan)

True

In [8]:
#Opposites
pd.notna(None)

False

In [9]:
pd.notna(np.nan)

False

In [10]:
pd.notnull(None)

False

In [11]:
pd.notnull(np.nan)

False

These functions also work with Series and DataFrames:

In [13]:
pd.isnull(pd.Series([1,2,None,4,5]))

0    False
1    False
2     True
3    False
4    False
dtype: bool

In [15]:
pd.notnull(pd.Series([1,2,3,None,6]))

0     True
1     True
2     True
3    False
4     True
dtype: bool

In [16]:
pd.isnull(pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
}))


,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


In [17]:
pd.Series([1, 2, np.nan]).count()  #count doesnt incluse null values; gives how many not null values

2

In [18]:
pd.Series([1, 2, np.nan]).sum()         #includes null values; gives total entries

3.0

### Filtering missing data
As we saw with numpy, we could combine boolean selection + pd.isnull to filter out those nans and null values

In [20]:
s=pd.Series([1,2,3,None,4,None])
s

0    1.0
1    2.0
2    3.0
3    NaN
4    4.0
5    NaN
dtype: float64

In [21]:
pd.isnull(s)

0    False
1    False
2    False
3     True
4    False
5     True
dtype: bool

In [22]:
pd.notnull(s)

0     True
1     True
2     True
3    False
4     True
5    False
dtype: bool

In [27]:
pd.notnull(s).sum()

4

In [28]:
pd.isnull(s).sum()

2

In [29]:
s[pd.notnull(s)]

0    1.0
1    2.0
2    3.0
4    4.0
dtype: float64

In [30]:
s.isnull()

0    False
1    False
2    False
3     True
4    False
5     True
dtype: bool

In [31]:
s.notnull()

0     True
1     True
2     True
3    False
4     True
5    False
dtype: bool

### Dropping null values

In [ ]:
#dropping null values in series

In [32]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    4.0
5    NaN
dtype: float64

In [33]:
s.dropna()

0    1.0
1    2.0
2    3.0
4    4.0
dtype: float64

### dropping null values in dataframes

You saw how simple it is to drop nas with a Series. But with DataFrames, there will be a few more things to consider, because you can't drop single values. You can only drop entire columns or rows

In [38]:
df=pd.DataFrame({
    'Column A':[1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110]})
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [39]:
df.isnull()

,Column A,Column B,Column C,Column D
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,True,True,False,False


In [40]:
df.isnull().sum()

Column A    2
Column B    1
Column C    1
Column D    0
dtype: int64

The default dropna behavior will drop all the rows in which any null value is present:

In [41]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In this case we're dropping rows. Rows containing null values are dropped from the DF. You can also use the axis parameter to drop columns containing null values:

In [42]:
df.dropna(axis=1)

,Column D
0,5
1,8
2,34
3,110


In this case, any row or column that contains at least one null value will be dropped. Which can be, depending on the case, too extreme. You can control this behavior with the how parameter. Can be either 'any' or 'all':

In [44]:
df2=pd.DataFrame({'Column A': [1, np.nan, 30],
    'Column B': [2, np.nan, 31],
    'Column C': [np.nan, np.nan, 100]})
df2

,Column A,Column B,Column C
0,1.0,2.0,NaN
1,NaN,NaN,NaN
2,30.0,31.0,100.0


In [45]:
df2.dropna(how='all')             #‘any’ : If any NA values are present, drop that row or column.
                                 #‘all’ : If all values are NA, drop that row or column.

,Column A,Column B,Column C
0,1.0,2.0,NaN
2,30.0,31.0,100.0


In [46]:
df2.dropna(how='any')

,Column A,Column B,Column C
2,30.0,31.0,100.0


In [47]:
df.dropna(how='any')

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In [48]:
df.dropna(how='all')

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


You can also use the thresh parameter to indicate a threshold (a minimum number) of non-null values for the row/column to be kept:

In [49]:
df.dropna(thresh=3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [51]:
df.dropna(thresh=3,axis=1)

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


### Filling null values
Sometimes instead than dropping the null values, we might need to replace them with some other value. This highly depends on your context and the dataset you're currently working. Sometimes a nan can be replaced with a 0, sometimes it can be replaced with the mean of the sample, and some other times you can take the closest value. Again, it depends on the context. We'll show you the different methods and mechanisms and you can then apply them to your own problem.

In [52]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    4.0
5    NaN
dtype: float64

In [53]:
#FILLING WITH ARBITARY VALUES
s.fillna(0)

0    1.0
1    2.0
2    3.0
3    0.0
4    4.0
5    0.0
dtype: float64

In [54]:
s.fillna(1)

0    1.0
1    2.0
2    3.0
3    1.0
4    4.0
5    1.0
dtype: float64

In [55]:
s.fillna(s.mean())

0    1.0
1    2.0
2    3.0
3    2.5
4    4.0
5    2.5
dtype: float64

In [56]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    4.0
5    NaN
dtype: float64

### Filling nulls with contiguous (close) values

The method argument is used to fill null values with other values close to that null one:

In [57]:
s.fillna(method='ffill')           #ffill: propagate last valid observation forward to next valid.

0    1.0
1    2.0
2    3.0
3    3.0
4    4.0
5    4.0
dtype: float64

In [58]:
s.fillna(method='bfill')          #backfill / bfill: use next valid observation to fill gap

0    1.0
1    2.0
2    3.0
3    4.0
4    4.0
5    NaN
dtype: float64

This can still leave null values at the extremes of the Series/DataFrame

In [59]:
pd.Series([np.nan, 3, np.nan, 9]).fillna(method='ffill')

0    NaN
1    3.0
2    3.0
3    9.0
dtype: float64

In [60]:
pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method='bfill')

0    1.0
1    3.0
2    3.0
3    NaN
4    NaN
dtype: float64

### Filling null values on DataFrames
The fillna method also works on DataFrames, and it works similarly. The main differences are that you can specify the axis (as usual, rows or columns) to use to fill the values (specially for methods) and that you have more control on the values passed:

In [61]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [62]:
df.fillna({'Column A': 0 , 'Column B':99 , 'Column C':df['Column C'].mean()})

,Column A,Column B,Column C,Column D
0,1.0,2.0,47.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,99.0,100.0,110


In [64]:
df.fillna(method='ffill' , axis=0)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [65]:
df.fillna(method='ffill' , axis=1)

,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0


### Checking if there are NAs
The question is: Does this Series or DataFrame contain any missing value? The answer should be yes or no: True or False. How can you verify it?

Example 1: Checking the length

If there are missing values, s.dropna() will have less elements than s:

In [69]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    4.0
5    NaN
dtype: float64

In [68]:
s.dropna().count()

4

In [70]:
missing_values = len(s.dropna()) != len(s)       #check if length of s after dropping nan values is less than original s
missing_values

True

In [71]:
len(s)

6

In [72]:
s.count()      #count does not include nan values

4

In [74]:
missing_values= len(s)!= s.count()
missing_values

True